In [3]:
import os


In [4]:
os.chdir("../") 

In [5]:
pwd

'd:\\chicken-diseases-classification'

In [12]:
!pip install joblib


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [35]:
from src.chicken_diseases_classification.constants import *
from src.chicken_diseases_classification.utils.common import read_yaml, create_directories

In [39]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_url=config.source_url,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir
          )

          return data_ingestion_config

    

In [40]:
import os
import urllib.request as request
import zipfile
from src.chicken_diseases_classification import logger
from src.chicken_diseases_classification.utils.common import get_size
from box import Box
from box.exceptions import BoxValueError

In [41]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info :\n{header}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):

        """zip_file_path : str
        Extracts the zip file into the data directory
        Function returns None"""

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [42]:
#creating the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-01-03 13:26:10,146: INFO: common :yaml file: config\config.yaml loaded sucessfully]
[2025-01-03 13:26:10,153: INFO: common :yaml file: params.yaml loaded sucessfully]
[2025-01-03 13:26:10,153: INFO: common :Created directory at artifacts]
[2025-01-03 13:26:10,156: INFO: common :Created directory at artifacts/data_ingestion]
[2025-01-03 13:26:28,096: INFO: 1897419661 :artifacts/data_ingestion/data.zip download! with following info :
Connection: close
Content-Length: 25022063
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7109e4947e2629e09308291ef3a6127aa1d9aa81a2f37b54e6e73c39045422a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD28:D2F73:832D0C:C149BE:67779497
Accept-Ranges: bytes
Date: Fri, 03 Jan 2025 07:41:13 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10228-MAA
X-Cache: MISS
